In [ ]:
# Workflow: Outage Incremental Error Prediction

This workflow notebook processes incremental device status changes and uses AI to predict and generate telemetry errors for devices with issues. It demonstrates the predictive capabilities of the self-healing networks system.

## Overview
- **Purpose**: Predict and generate telemetry errors for devices with issues
- **Input**: Incremental device status changes
- **Process**: AI-powered error prediction and generation
- **Output**: Device status triage table with predicted errors


In [ ]:
## Step 1: Setup and Load Incremental Data

Load incremental changes from the device status table using Delta Lake's change data feed. This captures only the new or updated records since the last processing run.


In [0]:
from pyspark.sql.functions import concat, col, lit
import mlflow
from yaml import safe_load

with open('../params.yml', 'r') as f:
      params = safe_load(f)

CATALOG = params.get('data_params')['catalog']
SCHEMA = params.get('data_params')['schema']



In [ ]:
## Step 2: Generate Telemetry Errors with AI

Use AI to generate realistic telemetry error messages for devices with "Telemetry Issue" status. The AI creates contextually appropriate error messages based on the device type and typical failure patterns.


In [0]:
inc_status = (spark.read.option("readChangeData", "true")
      .option('startingVersion', '3')
      .table(f"{CATALOG}.{SCHEMA}.device_status"))

inc_status.createOrReplaceTempView("inc_status_view")

display(inc_status)

In [ ]:
## Step 3: Save Triage Results

Save the generated telemetry errors to the device status triage table. This creates a record of all devices that need attention along with their specific error messages for further processing by the triage system.


In [0]:
triage_devices = spark.sql("""
    WITH tablestatuserrors AS (
        SELECT *, 
               ai_query('databricks-gemma-3-12b', 
                        CONCAT('what is a telemetry error for the following mobile tower device? Please be extremely concise and give only the example telemetry error, preferably 1 sentence max.', device)
               ) AS telemetry_error 
        FROM inc_status_view 
        WHERE status = 'Telemetry Issue'
    )
    SELECT tower_id, device_id, device, status, event_timestamp, telemetry_error
    FROM tablestatuserrors
""")
# triage_devices.write.mode('append').insertInto(f"{CATALOG}.{SCHEMA}.device_status_triage")

In [0]:
display(triage_devices)

In [0]:
triage_devices.write.mode('append').insertInto(f"{CATALOG}.{SCHEMA}.device_status_triage")